In [ ]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving audio.wav to audio.wav


In [1]:
num_speakers = 2 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'medium' #@param ['tiny', 'base', 'small', 'medium', 'large']

model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [2]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip uninstall speechbrain
!pip install speechbrain==0.5.16

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

Found existing installation: speechbrain 1.0.0
Uninstalling speechbrain-1.0.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/speechbrain-1.0.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/speechbrain/*
    /usr/local/lib/python3.10/dist-packages/tests/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/tests/test_optimizer.py
    /usr/local/lib/python3.10/dist-packages/tests/test_pipeline.py
    /usr/local/lib/python3.10/dist-packages/tests/utils.py
Proceed (Y/n)? Y
  Successfully uninstalled speechbrain-1.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 4.5 MB/s eta 0:00:00


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [7]:
path="/content/audio.wav"

In [8]:

if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

In [5]:
!sudo apt install portaudio19-dev
!pip install pyaudio
import pyaudio
!pip install pydub
!pip install -qq https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip
!pip install transformers
!pip install -q googletrans==4.0.0-rc1
#!pip install --upgrade tensorflow
#!pip install whisper
#!sudo apt update && sudo apt install ffmpeg
#!pip install -U openai-whisper
# !apt update
# !apt install ffmpeg -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (679 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

In [6]:
from googletrans import Translator
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random
import os
import json
import contextlib
import wave
from datetime import timedelta
import numpy as np
import pyaudio
import soundfile as sf
from pydub import AudioSegment
from pyannote.core import Segment
from sklearn.cluster import AgglomerativeClustering
import re
import whisper

In [9]:
from pydub import AudioSegment
audio = AudioSegment.from_file(path, format="wav")
channels = audio.channels
print("Number of channels:", channels)

if channels > 1:
    audio = audio.set_channels(1)
    audio.export('mono_audio.wav', format="wav")
    path = 'mono_audio.wav'

Number of channels: 2


In [10]:
model = whisper.load_model(model_size)

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 103MiB/s]


In [11]:
result = model.transcribe(path)
segments = result["segments"]

In [12]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [13]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]

  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [14]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [15]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [16]:

def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [17]:
def generate_name():
    first_names = ["Emma", "Liam", "Ava", "Noah", "Sophia", "William", "Isabella", "James", "Mia", "Benjamin",
                   "Charlotte", "Oliver", "Amelia", "Evelyn", "Henry", "Harper", "Ella", "Alexander", "Abigail", "Michael"]
    last_names = ["Smith", "Johnson", "Brown", "Garcia", "Miller", "Davis", "Rodriguez", "Martinez", "Hernandez", "Lopez",
                  "Gonzalez", "Perez", "Taylor", "Anderson", "Wilson", "Jackson", "White", "Harris", "Martin", "Thompson"]

    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    return f"{first_name} {last_name}"

def translate_to_telugu(text):
    translator = Translator()
    translation = translator.translate(text, src='en', dest='te')
    return translation.text

f = open("transcript.txt", "w", encoding='utf-8')
speakers = []
distinct_speakers = []
for (i, segment) in enumerate(segments):
    if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
        f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
        temp1 = time(segment["start"])
        temp2 = time(segment["end"])
        speakers.append(segment["speaker"])
    f.write(segment["text"][1:] + ' ')
    temp = set(speakers)
    for x in temp:
        if x not in distinct_speakers:
            distinct_speakers.append(segment["speaker"])
            distinct_speakers.append(segment["start"])
            distinct_speakers.append(segment["end"])

print("speakers are ::", speakers)
print("ds are ::", distinct_speakers)
f.close()

print(open("transcript.txt", 'r', encoding='utf-8').read())

print()

pattern = re.compile(r'SPEAKER (\d+) 0:(\d\d):(\d\d)\n(.*)')

with open("transcript.txt", "r", encoding='utf-8') as f:
    content = f.read()


dialogues = {}


for match in re.finditer(pattern, content):
    speaker = "SPEAKER " + str(match.group(1))
    dialogue = match.group(4)


    if speaker not in dialogues:
        dialogues[speaker] = []


    dialogues[speaker].append(dialogue)

data, samplerate = sf.read(path)
print("the sample rate is :", samplerate)
audio_file = AudioSegment.from_file(path, format="wav")
speaker_names = []
val = len(distinct_speakers) // 3

for i in range(val):
    j = 1 + i * 3
    start_time_str = str(distinct_speakers[j])
    end_time_str = str(distinct_speakers[j + 1])
    start_time = float(start_time_str)
    end_time = float(end_time_str)
    segment = audio_file[start_time * 1000:end_time * 1000]

    try:
        p = pyaudio.PyAudio()
        device_index = p.get_default_output_device_info()['index']
        stream = p.open(output_device_index=device_index, format=pyaudio.paInt16, channels=2, rate=35000, output=True)
        stream.write(segment.raw_data)
        stream.stop_stream()
        stream.close()
        p.terminate()
    except OSError as e:
        print(f"Skipping audio playback: {e}")

    name = generate_name()
    speaker_names.append(name)
    export_file = name + ".mp3"
    segment.export(export_file, format="mp3")

for i in range(val):
    distinct_speakers[i * 3] = speaker_names[i]

print(distinct_speakers)
print(speaker_names)

i = 0

for speaker, speaker_dialogues in dialogues.items():
    with open(speaker_names[i] + ".txt", "w", encoding='utf-8') as f:
        f.write(speaker_names[i] + ":" + "\n")
        f.write("\n".join(speaker_dialogues))
    i = i + 1

with open("transcript.txt", "r", encoding='utf-8') as f:
    transcript_text = f.read()
translated_transcript = translate_to_telugu(transcript_text)
with open("transcript_telugu.txt", "w", encoding='utf-8') as f:
    f.write(translated_transcript)

model_name = "pszemraj/led-base-book-summary"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
input_ids = tokenizer.encode(transcript_text, return_tensors="pt")
summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, max_length=160, min_length=56, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

with open("summary.txt", "w", encoding='utf-8') as f:
    f.write(summary)

translated_summary = translate_to_telugu(summary)
with open("summary_telugu.txt", "w", encoding='utf-8') as f:
    f.write(translated_summary)

for m in range(len(speaker_names)):
    if (os.path.exists(speaker_names[m] + ".txt")):
        with open(speaker_names[m] + ".txt", "r", encoding='utf-8') as f:
            text = f.read()
        input_ids = tokenizer.encode(text, return_tensors="pt")
        summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, max_length=142, min_length=56,
                                    early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        with open(speaker_names[m] + "summary" + ".txt", "w", encoding='utf-8') as f:
            f.write(summary)
    else:
        break

individual_summary_files_store = []
individual_script_files_store = []
actions_files_store = []

for name in speaker_names:
    actions_files_store.append(name + "actions.txt")
    individual_script_files_store.append(name + ".txt")
    individual_summary_files_store.append(name + "summary.txt")

transcript_file = 'transcript.txt'
summary_file = 'summary.txt'
actions_files = actions_files_store
individual_script_files = individual_script_files_store
individual_summary_files = individual_summary_files_store

speakers are :: ['SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1', 'SPEAKER 2', 'SPEAKER 1']
ds are :: ['SPEAKER 2', 0.0, 1.0, 'SPEAKER 1', 1.0, 4.24]

SPEAKER 2 0:00:00
King's restaurant. 
SPEAKER 1 0:00:01
Peter Chinn Good evening. I'm ringing about the job. I understand you have vacant? Catherine MacKinn Oh yes. Peter Chinn I'd like to find out a few more details, if I may. 
SPEAKER 2 0:00:10
Catherine MacKinn Yes, of course. Can I take your name? 
SPEAKER 1 0:00:11
Peter Chinn It's Peter Chinn. 
SPEAKER 2 0:00:13
Catherine MacKinn OK, Peter. Well, if you want to ask about the job and then if we're both still interested, we could arrange for you to come for an interview. 
SPEAKER 1 0:00:22
Peter Chinn Great, thanks. I'm afrai

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/648M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Input ids are automatically padded from 871 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 262 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 334 to 1024 to be a multiple of `config.attention_window`: 1024
